In [1]:
import netCDF4 as nc
from netCDF4 import Dataset
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# gsw oceanic toolbox: http://www.teos-10.org/pubs/Getting_Started.pdf
import gsw
from scipy.io import loadmat
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
from processing_func import check_coords, calc_N2_kappa, calc_hab, arctic_calchab, calc_N2_kappa_sorted, calc_sic
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

In [2]:
arctic_mix = "/Users/Lisanne/Documents/AI4ER/Mres/ArcticTurbulence/data/processed_data/arctic_mix.nc"
alberto_nc = "/Users/Lisanne/Documents/AI4ER/Mres/ArcticTurbulence/data/processed_data/alberto_ds.nc"
global_pkl = "/Users/Lisanne/Documents/AI4ER/Mres/ArcticTurbulence/data/input_microstructure.pkl"
global_nc = "/Users/Lisanne/Documents/AI4ER/Mres/ArcticTurbulence/data/L2_2D_snapshot_iy150_model_input.nc"
mosaic_nc = "/Users/Lisanne/Documents/AI4ER/Mres/ArcticTurbulence/data/processed_data/mosaic_ds.nc"
nice_nc = "/Users/Lisanne/Documents/AI4ER/Mres/ArcticTurbulence/data/processed_data/nice_ds.nc"
HM_nc = "/Users/Lisanne/Documents/AI4ER/Mres/ArcticTurbulence/data/processed_data/HM_ds.nc"
NN_nc = "/Users/Lisanne/Documents/AI4ER/Mres/ArcticTurbulence/data/L2_2D_snapshot_iy150_model_input.nc"

arctic_ds = xr.open_dataset(arctic_mix)
alberto_ds = xr.open_dataset(alberto_nc)
global_ds = pd.read_pickle(global_pkl)
global_nn = xr.open_dataset(global_nc)
mosaic_ds = xr.open_dataset(mosaic_nc)
nice_ds = xr.open_dataset(nice_nc)
HM_ds = xr.open_dataset(HM_nc)
NN_ds = xr.open_dataset(NN_nc)

In [3]:
# Assign the 'time' coordinate as a variable in the copy
nice_ds = nice_ds.rename({"time": "time_dim"})
nice_ds["time"] = nice_ds["time_dim"]

In [4]:
arctic_ds = arctic_ds.reset_coords(["latitude", "longitude", "time"])
mosaic_ds = mosaic_ds.reset_coords(["latitude", "longitude", "time"])
alberto_ds = alberto_ds.reset_coords(["latitude", "longitude", "time"])
nice_ds = nice_ds.reset_coords(["latitude", "longitude", "time"])
HM_ds = HM_ds.reset_coords(["latitude", "longitude", "time"])

In [5]:
# Bathymetry dataset
GEBCO_ds = "/Users/Lisanne/Documents/AI4ER/Mres/ArcticTurbulence/data/gebco_2022_n80.0_s63.0_w-170.0_e-130.0.nc"
bathy_ds = xr.open_dataset(GEBCO_ds)

In [6]:
# Sea ice fraction data
SI_HadISST = "/Users/Lisanne/Documents/AI4ER/Mres/ArcticTurbulence/data/SI-area/HadISST_ice.nc"
Hadi_SI = xr.open_dataset(SI_HadISST)

## Add all features and combine all datasets into one dataframe
The features and the plots are explained below

In [7]:
def convert_datetime(dataset):
    if type(dataset.time.values[0]) is not np.datetime64:
        dataset["time"] = xr.apply_ufunc(pd.to_datetime, dataset["time"], vectorize=True)
    return dataset

In [8]:
arctic_ds = convert_datetime(arctic_ds)
mosaic_ds = convert_datetime(mosaic_ds)

In [12]:
time_values = alberto_ds["time"].values
time_index = np.where(alberto_ds["time"].values == time_values[0])[0][0]

In [9]:
HM_kappa = HM_ds.squeeze()
arctic_ds = arctic_ds.squeeze()
arctic_ds = arctic_ds.transpose('depth', 'profile')
HM_kappa = HM_kappa.transpose('depth', 'profile')

nice_kappa = calc_N2_kappa_sorted(nice_ds)
mosaic_kappa = calc_N2_kappa_sorted(mosaic_ds)
HM_kappa = calc_N2_kappa_sorted(HM_kappa)
arctic_kappa = calc_N2_kappa_sorted(arctic_ds)
alberto_kappa = calc_N2_kappa_sorted(alberto_ds)

nice_kappa = calc_sic(nice_kappa, Hadi_SI)
alberto_kappa = calc_sic(alberto_kappa, Hadi_SI)
HM_kappa = calc_sic(HM_kappa, Hadi_SI)
arctic_kappa = calc_sic(arctic_kappa, Hadi_SI)
alberto_kappa = calc_sic(alberto_kappa, Hadi_SI)

arctic_final = arctic_calchab(arctic_kappa, bathy_ds)
nice_final = calc_hab(nice_kappa, bathy_ds)
mosaic_final = calc_hab(mosaic_kappa, bathy_ds)
HM_final = calc_hab(HM_kappa, bathy_ds)
alberto_final = calc_hab(alberto_kappa, bathy_ds)

IndexError: index 0 is out of bounds for axis 0 with size 0

In [9]:
# add variables of the cruise name
arctic_final["cruise"] = "ArcticMix"
nice_final["cruise"] = "NICE"
mosaic_final["cruise"] = "Mosaic"
alberto_final["cruise"] = "Alberto"
HM_final["cruise"] = "HM"

In [10]:
nice_fin = nice_final
nice_fin = nice_fin.rename({"time": "time-coord"})
nice_fin["time"] = nice_fin["time-coord"]
nice_fin["time"] = xr.DataArray(nice_fin["time"].values, dims='profile')
nice_selected = nice_fin.drop_vars(['time-coord', 'LATITUDE', "LONGITUDE"])

## Select variables from datasets

In [11]:
def select_variables(data, variables):
    """
    Select specific variables from the given dataset and reduce dimensions and coordinates to depth and profile.
    Args:
        data (xarray.Dataset): Input dataset containing variables.
        variables (list): List of variable names to select.
    Returns:
        xarray.Dataset: Dataset with selected variables and reduced dimensions/coordinates.
    """
    # Select the desired variables from the dataset
    selected_data = data[variables]
    # Reduce dimensions and coordinates to depth and profile
    selected_data = selected_data.squeeze().reset_coords(drop=True)
    return selected_data

In [30]:
# selected_columns = ["depth", "profile", "latitude", "longitude", "P", "S", "T", "Tu", "kappa", "log_N2", "log_kappa", "log_eps", "dTdz", "dSdz", "eps", "cruise", "hab", "Tu_label", "time"]
selected_columns = ["depth", "profile", "latitude", "longitude", "cruise", "S", "T", "eps", "log_N2", "dTdz", "dSdz", "hab", "Tu", "Tu_label", "time"]

arctic_f = select_variables(arctic_final, selected_columns)
alberto_f = select_variables(alberto_final, selected_columns)
mosaic_f = select_variables(mosaic_final, selected_columns)
HM_f = select_variables(HM_final, selected_columns)
nice_f = select_variables(nice_selected, selected_columns)

In [31]:
arctic_df = arctic_f.to_dataframe().reset_index()
HM_df = HM_f.to_dataframe().reset_index()
mosaic_df = mosaic_f.to_dataframe().reset_index()
alberto_df = alberto_f.to_dataframe().reset_index()
nice_df = nice_f.to_dataframe().reset_index()

In [32]:
combined_df = pd.concat([arctic_df, HM_df, mosaic_df, alberto_df, nice_df])
combined_nona = combined_df.copy()
combined_nona = combined_nona.dropna()
len(combined_nona)

627907

2101764 without
1307594 with P, S, T
1287803 with P, S, T, eps ## baseline
876875 with P, S, T, eps, logN2
876875 with P, S, T, eps, logN2, dTdz, dS,dz # all still aboard
867841 with P, S, T, eps, logN2, dTdz, dS,dz, hab
867841 with P, S, T, eps, logN2, dTdz, dS,dz, hab, Tu & Tu_label
862023 with P, S, T, eps, logN2, dTdz, dS,dz, hab, Tu & Tu_label and time

In [33]:
combined_nona.cruise.unique()

array(['ArcticMix', 'HM', 'Mosaic', 'Alberto', 'NICE'], dtype=object)

In [34]:
combined_nona.to_pickle('/Users/Lisanne/Documents/AI4ER/Mres/ArcticTurbulence/data/processed_data/ml_ready/2905all.pkl')

## Visualise this non na dataset

In [ ]:
full_df = pd.read_pickle('/Users/Lisanne/Documents/AI4ER/Mres/ArcticTurbulence/data/processed_data/ml_ready/2605all.pkl')

In [ ]:
fig = plt.figure(figsize=(6, 6))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.NorthPolarStereo())

# Add land, ocean, and borders
ax.add_feature(cfeature.OCEAN.with_scale('50m'))
ax.add_feature(cfeature.LAND.with_scale('50m'))

# Convert non-numeric cruise values to category codes
full_df["cruise_codes"] = full_df["cruise"].astype("category").cat.codes

ax.scatter(full_df["longitude"], full_df["latitude"], c=full_df["cruise_codes"],
           transform=ccrs.PlateCarree(), s=5)

#ax.scatter(full_df["longitude"], full_df["latitude"], c = full_df["cruise"], transform=ccrs.PlateCarree(), s=5)

# Add grid lines
gl = ax.gridlines(draw_labels=True, linestyle='--')
gl.xlabels_top = False
gl.ylabels_right = False
gl.bottom_labels = True  # Show x-axis labels at the bottom
gl.left_labels = True  # Show y-axis labels on the left

ax.set_extent([-180, 180, 65, 90], crs=ccrs.PlateCarree())
ax.set_title("Round Map with Grid Lines and Plot Points")
ax.legend(loc="center left")
plt.show()

In [ ]:
fig = plt.figure(figsize=(6, 6))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.NorthPolarStereo())

# Add land, ocean, and borders
ax.add_feature(cfeature.OCEAN.with_scale('50m'))
ax.add_feature(cfeature.LAND.with_scale('50m'))

# Convert non-numeric cruise values to category codes
combined_nona["cruise_codes"] = combined_nona["cruise"].astype("category").cat.codes

ax.scatter(combined_nona["longitude"], combined_nona["latitude"], c=combined_nona["cruise_codes"],
           transform=ccrs.PlateCarree(), s=5)

#ax.scatter(full_df["longitude"], full_df["latitude"], c = full_df["cruise"], transform=ccrs.PlateCarree(), s=5)

# Add grid lines
gl = ax.gridlines(draw_labels=True, linestyle='--')
gl.xlabels_top = False
gl.ylabels_right = False
gl.bottom_labels = True  # Show x-axis labels at the bottom
gl.left_labels = True  # Show y-axis labels on the left

ax.set_extent([-180, 180, 65, 90], crs=ccrs.PlateCarree())
ax.set_title("Round Map with Grid Lines and Plot Points")
ax.legend(loc="center left")
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Set up Seaborn style
sns.set(style='whitegrid')

# Define the depth values and cruise labels
depth_values = [combined_nona.loc[combined_nona['cruise'] == cruise, 'depth'] for cruise in combined_nona['cruise'].unique()]
labels = combined_nona['cruise'].unique()

# Plot a stacked histogram with labeled bars
plt.figure(figsize=(10, 6))
plt.hist(depth_values, bins=50, stacked=True, label=labels, alpha=0.7)

plt.xlabel('Depth')
plt.ylabel('Count')
plt.title('Stacked Histogram of Depth Values by Dataset')
plt.legend()

# Remove the right and top spines
sns.despine()

# Display the plot
plt.show()

## Visualise all the points

In [ ]:
total_profiles = len(HM_ds["longitude"]) + len(nice_ds["longitude"]) + len(mosaic_ds["longitude"]) + len(alberto_ds["longitude"]) + len(arctic_ds["longitude"])
print("total amount of profiles is: ", total_profiles)
fig = plt.figure(figsize=(6, 6))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.NorthPolarStereo())

# Add land, ocean, and borders
ax.add_feature(cfeature.OCEAN.with_scale('50m'))
ax.add_feature(cfeature.LAND.with_scale('50m'))

ax.scatter(HM_ds["longitude"], HM_ds["latitude"], transform=ccrs.PlateCarree(), color='red', s=5, label="HM")
ax.scatter(nice_ds["longitude"], nice_ds["latitude"], transform=ccrs.PlateCarree(), color='blue', s=5, label="Nice")
ax.scatter(mosaic_ds["longitude"], mosaic_ds["latitude"], transform=ccrs.PlateCarree(), color='green', s=5, label="Mosaic")
ax.scatter(alberto_ds["longitude"], alberto_ds["latitude"], transform=ccrs.PlateCarree(), color='yellow', s=5, label="Alberto")
ax.scatter(arctic_ds["longitude"], arctic_ds["latitude"], transform=ccrs.PlateCarree(), color='purple', s=5, label="ArcticMix")

# Add grid lines
gl = ax.gridlines(draw_labels=True, linestyle='--')
gl.xlabels_top = False
gl.ylabels_right = False
gl.bottom_labels = True  # Show x-axis labels at the bottom
gl.left_labels = True  # Show y-axis labels on the left

ax.set_extent([-180, 180, 65, 90], crs=ccrs.PlateCarree())
ax.set_title("Round Map with Grid Lines and Plot Points")
ax.legend(loc="center left")
plt.show()

In [ ]:
total_profiles = len(HM_ds["longitude"]) + len(nice_ds["longitude"]) + len(mosaic_ds["longitude"]) + len(alberto_ds["longitude"]) + len(arctic_ds["longitude"])
print("total amount of profiles is: ", total_profiles)
fig = plt.figure(figsize=(20, 20))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())
ax.scatter(HM_ds["longitude"], HM_ds["latitude"], transform=ccrs.PlateCarree(), color='red', s=5, label = "HM")
ax.scatter(nice_ds["longitude"], nice_ds["latitude"], transform=ccrs.PlateCarree(), color='blue', s=5, label = "Nice")
ax.scatter(mosaic_ds["longitude"], mosaic_ds["latitude"], transform=ccrs.PlateCarree(), color='green', s=5, label = "Mosaic")
ax.scatter(alberto_ds["longitude"], alberto_ds["latitude"], transform=ccrs.PlateCarree(), color='yellow', s=5, label = "Alberto")
ax.scatter(arctic_ds["longitude"], arctic_ds["latitude"], transform=ccrs.PlateCarree(), color='purple', s=5, label = "ArcticMix")

# Add land, ocean, and borders
ax.add_feature(cfeature.LAND.with_scale('50m'))
ax.add_feature(cfeature.OCEAN.with_scale('50m'))

#ax.set_extent([-180, 180, -90, 90], crs=ccrs.PlateCarree())
ax.set_title("Map with Land Boundaries and Plot Points")
ax.legend()
plt.show()

In [ ]:
# Select the depth variable from each dataset
depth1 = arctic_ds['depth'].sel()
depth2 = alberto_ds['depth'].sel()
depth3 = nice_ds['depth'].sel()
depth4 = mosaic_ds['depth'].sel()
depth5 = HM_ds['depth'].sel()

# Concatenate the depth values into a single array
depth_values = np.concatenate([depth1.values, depth2.values, depth3.values, depth4.values, depth5.values])

# Plot a histogram of the combined depth values
plt.hist(depth_values, bins=50)
plt.xlabel('Depth')
plt.ylabel('Count')
plt.title('Histogram of Depth Values')
plt.show()

In [ ]:
import seaborn as sns

# Select the depth variable from each dataset
depth1 = arctic_ds['depth'].sel()
depth2 = alberto_ds['depth'].sel()
depth3 = nice_ds['depth'].sel()
depth4 = mosaic_ds['depth'].sel()
depth5 = HM_ds['depth'].sel()

# Combine the depth values into a single array
depth_values = np.concatenate([depth1.values, depth2.values, depth3.values, depth4.values, depth5.values])

# Create a list of labels for each dataset
labels = ['Arctic', 'Alberto', 'Nice', 'Mosaic', 'HM']

# Set up Seaborn style
sns.set(style='whitegrid')

# Plot a stacked histogram with labeled bars
plt.figure(figsize=(10, 6))
plt.hist([depth1, depth2, depth3, depth4, depth5], bins=50, stacked=True, label=labels, alpha=0.7)

plt.xlabel('Depth')
plt.ylabel('Count')
plt.title('Stacked Histogram of Depth Values by Dataset')
plt.legend()

# Remove the right and top spines
sns.despine()

# Display the plot
plt.show()

# From now on, pretend like nice_ds is the combined_ds


## Calculate features like in Mashayek et al. (2021)

In [ ]:
from scipy.interpolate import interp1d
from matplotlib import colors

In [ ]:
nice_kappa = calc_N2_kappa(nice_ds)
nice_kappa.kappa.plot(y = 'depth',norm = colors.LogNorm(vmin = 1e-7, vmax = 1e-1),cmap='viridis',figsize=(20,5))
plt.gca().invert_yaxis()
plt.show()

In [ ]:
nice_kappa.eps.plot(y = 'depth',norm = colors.LogNorm(vmin = 1e-10, vmax = 1e-4),cmap='viridis',figsize=(20,5))
plt.gca().invert_yaxis()
plt.show()

In [ ]:
HM_kappa = HM_ds.squeeze()
HM_kappa = HM_kappa.transpose('depth', 'profile')
HM = calc_N2_kappa(HM_kappa)
HM.kappa.plot(y = 'depth',norm = colors.LogNorm(vmin = 1e-7, vmax = 1e-4),cmap='viridis',figsize=(20,5))
plt.gca().invert_yaxis()
plt.show()

In [ ]:
HM.eps.plot(y = 'depth',norm = colors.LogNorm(vmin = 1e-11, vmax = 1e-7),cmap='viridis',figsize=(20,5))
plt.gca().invert_yaxis()
plt.show()

In [ ]:
alberto_kappa = calc_N2_kappa(alberto_ds)
alberto_kappa.kappa.plot(y = 'depth',norm = colors.LogNorm(vmin = 1e-5, vmax = 1e1),cmap='viridis',figsize=(20,5))
plt.gca().invert_yaxis()
plt.show()

In [ ]:
alberto_kappa.eps.plot(y = 'depth',norm = colors.LogNorm(vmin = 1e-7, vmax = 1e1),cmap='viridis',figsize=(20,5))
plt.gca().invert_yaxis()
plt.show()

In [ ]:
arctic_ds = arctic_ds.squeeze()
arctic = calc_N2_kappa_func(arctic_ds)
arctic.kappa.plot(y = 'depth',norm = colors.LogNorm(vmin = 1e-7, vmax = 1e-1),cmap='viridis',figsize=(20,5))
plt.gca().invert_yaxis()
plt.show()

In [ ]:
mosaic = calc_N2_kappa(mosaic_ds)
mosaic.kappa.plot(y = 'depth',norm = colors.LogNorm(vmin = 1e4, vmax = 1e7),cmap='viridis',figsize=(20,5))
plt.gca().invert_yaxis()
plt.show()

In [ ]:
mosaic.eps.plot(y = 'depth', cmap='viridis',figsize=(20,5))


## Hab

In [ ]:
from tqdm import tqdm

In [ ]:
def calc_hab_func(data):
    bathy_interp = bathy_ds.interp_like(data, method='nearest')
    n_depths = data.profile.shape[0]
    depth = np.zeros(n_depths)

    for i in tqdm(range(n_depths)):
        microlon = data.longitude[i].values.flatten()
        microlat = data.latitude[i].values.flatten()
        depth[i] = bathy_interp.elevation.sel(lon=microlon,lat=microlat, method='nearest')
    data['bathymetry'] = data.profile.copy(data=depth)
    
    data["hab"] = data.bathymetry + abs(data.depth)
    data["hab"] = data["hab"].where(data["hab"] <= 0, 0)  # Set positive values to zero
    return data

In [ ]:
alberto_hab = calc_hab_func(alberto_ds)
alberto_hab.bathymetry.plot()

In [ ]:
mosaic_hab = calc_hab_func(mosaic_ds)
mosaic_hab.bathymetry.plot()

In [ ]:
nice_hab = calc_hab_func(nice_ds)
nice_hab.bathymetry.plot()

In [ ]:
HM_hab = calc_hab_func(HM_ds)
HM_hab.bathymetry.plot()

In [ ]:
def arctic_calchab_func(data):
    # group data by the 'profile' dimension
    profile_groups = data.groupby('profile')

    bathy_interp = bathy_ds.interp_like(data, method='nearest')
    n_profiles = len(data.depth)

    profile = np.zeros(len(profile_groups))

    # loop over each group
    for i, (_, profile_data) in tqdm(enumerate(profile_groups)):
        microlat = profile_data.latitude.values.flatten()[0]
        microlon = profile_data.longitude.values.flatten()[0]
        profile[i] = bathy_interp.elevation.sel(lon=microlon, lat=microlat, method='nearest').values

    # Create a DataArray for bathymetry with the 'profile' dimension
    profile_arr = xr.DataArray(profile, coords=[range(len(profile_groups))], dims=['profile'])
    data['bathymetry'] = profile_arr
    data["hab"] = data.bathymetry + abs(data.depth)
    data["hab"] = data["hab"].where(data["hab"] <= 0, 0)  # Set positive values to zero
    return data

In [ ]:
arctic_hab = arctic_calchab_func(arctic_ds)
arctic_hab.bathymetry.plot()

## spare functions, from processing_func.py

In [ ]:
def check_coords(data):
    if "latitude" not in data.coords:
        data = data.set_coords("latitude")
    if "longitude" not in data.coords:
        data = data.set_coords("longitude")
    if "time" not in data.coords:
        data = data.set_coords("time")
    if "depth" not in data.coords:
        data["depth"] = data.depth
    return data

In [ ]:
def TS_derivative(dataset):
  dataset["dTdz"] = dataset.T.differentiate('depth')
  dataset['dSdz'] = dataset.S.differentiate('depth')
  return dataset

In [ ]:
def interpolate_pmid(dataset, variable):
    original_depth = dataset.depth  # Assuming you have access to the original depth values
    depth_old = np.arange(variable.shape[0])  # Depth values of the variable

    # Extend the depth array to include the boundary value
    depth_new = np.append(depth_old, original_depth[-1])

    # Extend the variable array to include the boundary value
    # new_array = np.column_stack((variable, variable[:, -1]))
    new_array = np.vstack((variable, variable[-1, :]))

    # Create a 1D interpolation function along the depth dimension
    interp_func = interp1d(depth_new, new_array, axis=0, kind='linear')
    # Interpolate the variable to match the original depth dimension
    var_interp = interp_func(original_depth)

    # Create a DataArray with explicit dimension names
    var_dataarray = xr.DataArray(var_interp, dims=('depth', 'profile'))
    return var_dataarray

In [ ]:
def interpolate_pmid_arctic(dataset, variable):
    original_depth = dataset.depth  # Assuming you have access to the original depth values
    depth_old = np.arange(variable.shape[0])  # Depth values of the variable

    # Extend the depth array to include the boundary value
    depth_new = np.append(depth_old, original_depth[-1])

    # Extend the variable array to include the boundary value
    # new_array = np.column_stack((variable, variable[:, -1]))
    new_array = np.vstack((variable, variable[-1, :]))

    # Create a 1D interpolation function along the depth dimension
    interp_func = interp1d(depth_new, new_array, axis=0, kind='linear')
    # Interpolate the variable to match the original depth dimension
    var_interp = interp_func(original_depth)

    # Create a DataArray with explicit dimension names
    var_dataarray = xr.DataArray(var_interp, dims=('depth', 'profile'))
    return var_dataarray

In [ ]:
def calc_N2_kappa_func(dataset, arctic="false"):
    """N2 and kappa both independent of epsilon

    Parameters
    ----------
    dataset : dataset
        Microstructure dataset, where insitu temperature is named as "T" in
        degrees Celcius.
        Salinity is named as "S" in .., and depth is called "depth" in meters.
    """
    S = dataset.S
    P = dataset.P
    lon = dataset.longitude.squeeze()
    lat = dataset.latitude.squeeze()
    T = dataset.T
    eps = dataset.eps
    z = dataset.depth
    # Add a dummy axis to pressure (P) to match the shape of other variables
    if T.shape != P.shape:
        # Project the lower-dimensional variable onto the target dimension
        P = np.expand_dims(P, axis=1)

    # convert measured T to potential T, which is seen as temperature now
    # potential temperature is the temperature a water parcel would have 
    # if it were brought to the surface adiabatically (no pressure effects)
    dataset = dataset.rename({"T": "insituT"})
    dataset["T"] = gsw.conversions.pt_from_t(S, T, P, p_ref=0)
    dataset['rho'] = gsw.rho(S, T, P)
    dataset["SA"] = gsw.SA_from_SP(S, P, lon, lat)
    dataset["CT"] = gsw.CT_from_t(dataset.SA, T, P)

    # calculate the turner angle
    # The values of Turner Angle Tu and density ratio Rrho are calculated
    # at mid-point pressures, p_mid.
    # https://teos-10.org/pubs/gsw/html/gsw_Turner_Rsubrho.html
    [Tu, Rsubrho, p_mid] = gsw.Turner_Rsubrho(dataset.SA, dataset.CT, P)

    dataset["Tu"] = interpolate_pmid(dataset, Tu)
    dataset["Rsubrho"] = interpolate_pmid(dataset, Rsubrho)

    # Calculate N^2 using gsw_Nsquared
    # https://teos-10.org/pubs/gsw/html/gsw_Nsquared.html
    [N2, p_mid] = gsw.Nsquared(SA=dataset["SA"], CT=dataset["CT"], p=P,
                               lat=dataset["latitude"])
    dataset["N2"] = interpolate_pmid(dataset, N2)

    # calculate kappa like in Mashayek et al, 2022
    # assume chi is 0.2 in standard turbulence regime
    dataset['kappa'] = 0.2*dataset.eps/dataset.N2
    # assume mixing efficiency of 1 in double diffusion regime
    dataset["kappa_AT"] = dataset.eps/dataset.N2

    dataset["log_N2"] = np.log10(dataset.N2)
    dataset["log_kappa"] = np.log10(dataset.kappa)

    dataset = TS_derivative(dataset)
    return dataset